In [ ]:
import pandas as pd
import os
from FinetuneCNN import Finetune
import numpy as np
import utils
import configure
import tensorflow as tf
import bot

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
os.environ["CUDA_VISIBLE_DEVICES"]="7"

pjoin = os.path.join

# This code will fine-tune with best hyperparams (saved in all_folds.csv) again, then test it on test set.
# the model's weight from fine-tuning in this code
# will be saved in the same finetune_weight_path of fine-tuning time. 
# (the new dir will be created for logging only)

In [ ]:
# define path location here
dirpath = './logs'

# define list of finish finetune path here
path_list = '''2020-10-15_10:14:21
2020-10-15_10:18:37
2020-10-15_12:11:49
2020-10-15_12:21:06
2020-10-15_12:32:58
2020-10-15_12:56:54
2020-10-15_13:02:38
2020-10-15_13:05:55'''

path_list = path_list.split('\n')
all_results_path_list = [pjoin(dirpath, p) for p in path_list]

# in case of various weights location
path_check_list = []

In [ ]:
found_error = False
try:
    for all_results_path in all_results_path_list:
        print('📍 testing .. all_results_path:', all_results_path, '📍')

        nfolds = 1

        results = []
        test_results = []
        done_subjects = []

        t = pjoin(all_results_path, 'test_results.csv')

        # read all folds results
        df = pd.read_csv(pjoin(all_results_path, 'all_folds.csv'))

        if type(df['random_round'].values[0]) != np.int64:
            df = df.loc[df['random_round']!='-']

        df = df.reset_index()

        df[['random_round', 'K', 'nepochs', 'lr', 'loss', 'W', 'N1', 'N2', 'N3', 'REM', 'MF1']] = df[['random_round', 
                                                                                                      'K', 'nepochs', 'lr', 'loss',
                                                                                                      'W', 'N1', 
                                                                                                      'N2', 'N3', 
                                                                                                      'REM', 'MF1']].apply(pd.to_numeric)

        _, idx = np.unique(df['subj_id'].values, return_index=True)
        subjects = df['subj_id'].values[np.sort(idx)]

        pretrain_path = df['pretrain_path'].values[0]
        channel = df['channel'].values[0]
        max_epochs = np.max(df['nepochs'].values)
        print('pretrain_path:', pretrain_path)
        print('max_epochs:', max_epochs)

        # average results from all random rounds in each subject & each set of hyperparams
        results_df = pd.DataFrame(df.groupby(['subj_id', 'K', 'nepochs', 'lr'])['loss', 
                                                                         'W', 'N1', 
                                                                         'N2', 'N3', 
                                                                         'REM', 'MF1'].mean())

        # get lowest loss of each subject
        res_all = results_df.loc[results_df.groupby(['subj_id'])['loss'].idxmin()]
        res_all = res_all.reset_index()

        all_kappa_separate_round, all_mf1_separate_round, all_f1_separate_round, subjects_separate_round = [], [], [], []
        all_acc_separate_round = []
        all_lr_separate_round, all_nepochs_path_separate_round  = [], []

        print('subjects:', subjects)
        for subject_id in subjects:

            if not subject_id in done_subjects:

                # select only current subject
                res = res_all.loc[res_all['subj_id'] == subject_id]
                if len(res) == 0:
                    print('Subject:', subject_id, 'Not Found.')
                    continue

                if len(res) > 1:
                    raise Exception('More than 1 best hyperparams found.')

                if len(res) == 1:
                    # get best hyperparams of this subject
                    best_k = res['K'].values[0]
                    best_nepochs = res['nepochs'].values[0]
                    best_lr = res['lr'].values[0]
                    print('best_k:', best_k, 'best_nepochs:', best_nepochs, 'best_lr:', best_lr)

                    # get finetune weight path of best hyperparams of this subject
                    best_df = df.loc[(df['subj_id'] == subject_id) & 
                                     (df['K'] == best_k) & 
                                     (df['nepochs'] == best_nepochs) &
                                     (df['lr'] == best_lr)
                                    ]
                    print(best_df)

                    # save all rounds results tested with best params of this subject
                    best_df.to_csv(pjoin(all_results_path, 'best_from_subject_'+str(subject_id)+'.csv'), index=False)

                    # test with best hyperparams
                    loss_all_round, acc_all_round, f1_all_round, kappa_all_round = [], [], [], []
                    finetune_weight_path_all = []
                    col = ['subj_id', 'K', 'lr', 'nepochs','pretrain_path', 'loss',
                           'acc', 'W', 'N1', 'N2', 'N3', 'REM', 'MF1']
                    for random_time in range(0, nfolds):
                        print('testing random round:', random_time)
                        random_seed = (random_time + 1) * 3

                        best_df_row = best_df.loc[best_df['random_round']==random_time]
                        finetune_weight_path = best_df_row['finetune_weight_path'].values[0]

                        if pretrain_path == np.nan or str(pretrain_path) == 'nan':
                            print('None pretrain path: use the saved weights, not fine-tune again.')

                            # use ds_name from configure.finetune_dataset, but channel from all_folds.csv
                            finetune = Finetune(best_k, 
                                                subject_id = subject_id,
                                                model_pretrain_path = pretrain_path,
                                                finetune_weight_path = finetune_weight_path,
                                                seed = random_seed,
                                                nepochs = [best_nepochs],
                                                lr = best_lr,
                                                ds_name = configure.finetune_dataset,
                                                channel = channel
                                               )
                            finetune.init_model_ops()

                            loss, acc, f1, kappa = finetune.predict(finetune_weight_path, best_epoch=best_nepochs)
                        else:
                            w_exist = os.path.exists(finetune_weight_path)
                            print('finetune_weight_path:', finetune_weight_path, w_exist)  

                            for path_check in path_check_list:
                                if not w_exist:
                                    path_tmp = pjoin(path_check,
                                                     finetune_weight_path[7:])
                                    print('Checking path_tmp:', path_tmp)
                                    w_exist = os.path.exists(path_tmp)
                                    if w_exist: 
                                        finetune_weight_path = path_tmp
                                        print('finetune_weight_path changed:', finetune_weight_path, w_exist) 
                                else:
                                    break


                            if not w_exist:
                                print('‼️ ALERT finetune_weight_path not found:', finetune_weight_path)
                                bot.sendMsg('⚠️ NO PATH Found, created new one:', finetune_weight_path, 
                                           '@ all_results_path:', all_results_path)
                                os.makedirs(finetune_weight_path)
                                
                            # use ds_name from configure.finetune_dataset, but channel from all_folds.csv
                            finetune = Finetune(best_k, 
                                                subject_id = subject_id,
                                                model_pretrain_path = pretrain_path,
                                                finetune_weight_path = finetune_weight_path,
                                                seed = random_seed,
                                                nepochs = [best_nepochs],
                                                lr = best_lr,
                                                ds_name = configure.finetune_dataset,
                                                channel = channel
                                               )
                            finetune.init_model_ops()
                            
                            wpath = pjoin(finetune_weight_path, "model_iter"+str(best_nepochs)+".ckpt.index")
                            if not os.path.exists(wpath):
                                raise Exception('Fine-tune weight not found: ' + str(wpath))
                            else:
                                loss, acc, f1, kappa = finetune.predict(finetune_weight_path, best_epoch=best_nepochs) 
                                
                        all_lr_separate_round.append(best_lr)
                        all_nepochs_path_separate_round.append(best_nepochs)
                        loss_all_round.append(loss)
                        acc_all_round.append(acc)
                        f1_all_round.append(f1)
                        kappa_all_round.append(kappa[0])
                        all_kappa_separate_round.append(kappa[0])
                        all_f1_separate_round.append(f1[0])
                        all_mf1_separate_round.append(np.mean(f1[0]))
                        all_acc_separate_round.append(acc[0])
                        subjects_separate_round.append(subject_id)
                        finetune_weight_path_all.append(finetune_weight_path)
                        col.append('finetune_weight_path:round'+str(random_time))

                        del finetune

                    mean_loss = np.mean(loss_all_round)
                    mean_acc = np.mean(acc_all_round)
                    mean_f1_per_class = np.mean(f1_all_round, axis=0)[0]
                    mf1 = np.mean(mean_f1_per_class)
                    print('mean_f1_per_class:', mean_f1_per_class, 'mean_f1_per_class:', mf1)

                    test_res = [subject_id, best_k, best_lr, best_nepochs, pretrain_path, mean_loss, mean_acc]
                    test_res.extend(mean_f1_per_class)
                    test_res.append(mf1)
                    test_res.extend(finetune_weight_path_all)
                    print(test_res)

                    test_results.append(test_res)

                    df2 = pd.DataFrame(test_results, columns=col)
                    print(df2)
                    df2.to_csv(pjoin(all_results_path, 'test_results.csv'), index=False)

                    del test_res, df2

        all_f1_separate_round = np.array(all_f1_separate_round)
        print(all_f1_separate_round.shape)

        df_kappa = pd.DataFrame({'subject_id': subjects_separate_round, 
                                 'K': [best_k] * len(subjects_separate_round),
                                 'lr': all_lr_separate_round,
                                 'nepochs': all_nepochs_path_separate_round,
                                 'pretrain_path': [pretrain_path] * len(subjects_separate_round),
                                 'kappa': all_kappa_separate_round,
                                 'acc': all_acc_separate_round,
                                 'f1_W': all_f1_separate_round[:,0],
                                 'f1_N1': all_f1_separate_round[:,1],
                                 'f1_N2': all_f1_separate_round[:,2],
                                 'f1_N3': all_f1_separate_round[:,3],
                                 'f1_R': all_f1_separate_round[:,4],
                                 'mf1': all_mf1_separate_round 
                                })
        df_kappa.to_csv(pjoin(all_results_path, 'test_results_separate_random_round.csv'), index=False)
        print(df_kappa)

        del df, results, test_results, done_subjects, df_kappa
        del all_kappa_separate_round, all_mf1_separate_round, all_f1_separate_round, subjects_separate_round
        del all_acc_separate_round, all_lr_separate_round, all_nepochs_path_separate_round
        del loss_all_round, acc_all_round, f1_all_round, kappa_all_round, finetune_weight_path_all

except Exception as e:
    print(e)
    bot.sendMsg('[🔴 DONE with ERROR] FinetuneAndTestOnBestHyperparams-List', path_list[0])
    found_error = True
    

In [ ]:
if not found_error:
    print('done')
    bot.sendMsg('[DONE] FinetuneAndTestOnBestHyperparams-List 🥳🥳🥳', path_list[0])